In [1]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [30]:
def get_transformation_matrix(animal):
    try:
        #url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        url = f'http://localhost:8000/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix

In [18]:
prep_id = 'DK52'

In [31]:
transformation = get_transformation_matrix(prep_id)
r = np.array(transformation['rotation'])
t = np.array(transformation['translation'])

In [32]:
print(r)
print(t)

[[ 0.90823119 -0.24441544 -0.05697743]
 [ 0.2488724   0.90471918  0.08611017]
 [ 0.03237074 -0.09804858  0.93659374]]
[[-15509.17500957]
 [-12625.10609468]
 [   -66.471405  ]]


In [33]:
person_id = 1 # me
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .filter(CenterOfMass.input_type == 'manual')\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section]

In [34]:
def brain_to_atlas_transform(
    brain_coord, r, t,
    brain_scale=(0.325, 0.325, 20),
    atlas_scale=(10, 10, 20)
):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates.
    
    The recorded r, t is the transformation from brain to atlas such that:
        t_phys = atlas_scale @ t
        atlas_coord_phys = r @ brain_coord_phys + t_phys
        
    Currently we erraneously have:
        t_phys = brain_scale @ t
    This should be fixed in the future.
    """
    brain_scale = np.diag(brain_scale)
    atlas_scale = np.diag(atlas_scale)

    # Transform brain coordinates to physical space
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    brain_coord_phys = brain_scale @ brain_coord
    
    # Apply affine transformation in physical space
    # t_phys = atlas_scale @ t
    # The following is wrong but it is what the rest of the code assumes
    # We need to fix it in the future
    # this gets flip flopped to above line
    t_phys = brain_scale @ t
    atlas_coord_phys = r @ brain_coord_phys + t_phys

    # Bring atlas coordinates back to atlas space
    atlas_coord = np.linalg.inv(atlas_scale) @ atlas_coord_phys

    return atlas_coord.T[0] # Convert back to a row vector

In [37]:
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    data.append([x, y, section])
    brain_coords = np.asarray([x, y, section])
    transformed = brain_to_atlas_transform(brain_coords, r, t)
    print(abbrev, x, y, section, np.around(transformed,1))
    x = transformed[0]
    y = transformed[1]
    section = transformed[2]
    com = CenterOfMass(
        prep_id=prep_id, structure=structure, x=x, y=y, section=section,
        created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned'
    )
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()



10N_L 45542.0 17755.0 218.0 [674.4 517.6 133.4]
10N_R 45714.0 17685.0 247.0 [676.7 522.  160.7]
12N 45596.0 18540.0 234.0 [667.9 543.9 147.1]
3N_L 34961.0 16285.0 232.0 [372.1 391.3 143.3]
3N_R 34749.0 16231.0 243.0 [365.  389.8 153.5]
4N_L 36247.0 16030.0 223.0 [413.1 392.6 135.9]
4N_R 36247.0 16056.0 247.0 [410.2 397.5 158.4]
5N_L 38765.0 19548.0 172.0 [465.3 507.6  83.9]
5N_R 38914.0 18314.0 309.0 [463.9 496.1 214.2]
6N_L 40686.0 19124.0 257.0 [515.7 525.3 165.2]
6N_R 40599.0 19425.0 220.0 [515.  527.1 130. ]
7N_L 44309.0 20809.0 307.0 [603.6 612.8 211.2]
7N_R 42465.0 22042.0 309.0 [539.1 634.5 210.2]
AP 46190.0 16965.0 230.0 [698.4 501.7 146.2]
Amb_L 44094.0 21914.0 172.0 [603.8 620.3  82.9]
DC_L 41986.0 17520.0 130.0 [581.3 466.8  49.5]
DC_R 42227.0 16113.0 343.0 [575.3 464.1 251.3]
IC 37176.0 9511.0 222.0 [492.4 208.3 145.9]
LC_L 39808.0 16939.0 188.0 [515.  442.1 103.6]
LC_R 39934.0 16233.0 278.0 [514.1 437.9 189. ]
LRt_L 47708.0 22042.0 183.0 [708.2 655.2  94.9]
LRt_R 47409.0 2

In [36]:
# Print atlas COMs for comparison

from utilities.sqlcontroller import SqlController

def get_atlas_centers(
        atlas_box_size=(1000, 1000, 300),
        atlas_box_scales=(10, 10, 20),
        atlas_raw_scale=10
):
    atlas_box_scales = np.array(atlas_box_scales)
    atlas_box_size = np.array(atlas_box_size)
    atlas_box_center = atlas_box_size / 2
    sqlController = SqlController('Atlas')
    atlas_centers = sqlController.get_centers_dict('Atlas')

    for structure, center in atlas_centers.items():
        # transform into the atlas box coordinates that neuroglancer assumes
        center = atlas_box_center + np.array(center) * atlas_raw_scale / atlas_box_scales
        atlas_centers[structure] = center

    return atlas_centers

get_atlas_centers()

No histology for Atlas
No scan run for Atlas


{'10N_L': array([703.341 , 547.9286, 135.6302]),
 '10N_R': array([703.341 , 547.9286, 164.3698]),
 '12N': array([697.227 , 567.0988, 149.25  ]),
 '3N_L': array([364.555 , 391.506 , 143.2358]),
 '3N_R': array([364.555 , 391.506 , 156.7642]),
 '4N_L': array([399.441  , 396.104  , 137.62875]),
 '4N_R': array([399.441  , 396.104  , 162.37125]),
 '5N_L': array([456.7992 , 502.07948,  81.9005 ]),
 '5N_R': array([456.7992 , 502.07948, 218.0995 ]),
 '6N_L': array([517.1597, 512.4922, 132.3135]),
 '6N_R': array([517.1597, 512.4922, 167.6865]),
 'AP': array([702.306 , 525.6525, 151.    ]),
 'Amb_L': array([612.44  , 603.826 ,  88.2695]),
 'Amb_R': array([612.44  , 603.826 , 211.7305]),
 'DC_L': array([583.2885, 456.4954,  42.084 ]),
 'DC_R': array([583.2885, 456.4954, 257.916 ]),
 'IC': array([496.7529, 251.693 , 150.    ]),
 'LC_L': array([496.19734, 442.8079 , 108.22515]),
 'LC_R': array([496.19734, 442.8079 , 191.77485]),
 'LRt_L': array([688.194, 646.863,  93.755]),
 'LRt_R': array([688.194,